### Classify based on basic image attributes

1. By orientation if not a square
2. Solely by image width or height
    - width groups:
        - w < 1000
        - w > 1000 
- By size

In [14]:
from PIL import Image
import shutil
import os

os.chdir(r'C:\Users\halloween\Downloads\explo')
root = os.getcwd()

landscape_path = os.path.join(root, "landscape")
portrait_path = os.path.join(root, "portrait")
square_path = os.path.join(root, "square")

os.makedirs(landscape_path, exist_ok=True)
os.makedirs(portrait_path, exist_ok=True)
os.makedirs(square_path, exist_ok=True)

for f in os.listdir():
    try:
        img = Image.open(f)
        width, height = img.size
        img.close()
    except IOError:
        continue

    f = os.path.join(root, f)
    if width - height > 10:
        shutil.move(f, landscape_path)
    elif height - width > 10:
        shutil.move(f, portrait_path)
    else:
        shutil.move(f, square_path)
        